In [66]:
from os import environ as ENV
from time import time as now
from multiprocessing import Pool
import json

import qbraid

import pandas as pd
from tqdm.notebook import tqdm

import sys
sys.path.append(ENV['PWD'])

from should_be_stdlib import *
from circuit_extra import *
from circuit_postprocess import *

In [2]:
# number of shots to use
SHOTS = 1000

# Load circuits

In [3]:
ang_circuits_df =     pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_angle_embed.xlsx', index_col=0)
amp_circuits_df =     pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed.xlsx', index_col=0)
amp_qft_circuits_df = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed_qft.xlsx', index_col=0)
[x.shape for x in [ang_circuits_df, amp_circuits_df, amp_qft_circuits_df]]

[(2926, 3), (2926, 3), (2926, 3)]

In [4]:
# transpile circuits ahead of time
# if IonQ-qiskit is installed, the ionq job submission expects qiskit circuits instead of the native ionq format!!
# but qiskit transpiler is much faster so it's fine
import qbraid

amp_qft_circuits  = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_qft_circuits_df['qasm2'])]
for i,c in enumerate(amp_qft_circuits): c.name = f'amp_qft_c{i}'

amp_circuits      = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_circuits_df['qasm2'])]
for i,c in enumerate(amp_circuits): c.name = f'amp_c{i}'

ang_circuits      = [qbraid.transpile(c, 'qiskit') for c in tqdm(ang_circuits_df['qasm2'])]
for i,c in enumerate(ang_circuits): c.name = f'ang_c{i}'


  0%|          | 0/2926 [00:00<?, ?it/s]

/home/user/work/quadrigems/.venv/lib/python3.11/site-packages/qbraid/_entrypoints.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

# IBMQ

## Connect to IBM API

In [185]:
from provider_ibmq import IBMQ
ibmq = IBMQ(
    ibm_api_key := open(ENV['HOME'] + '/work/api-keys/IBM_API_KEY').readline().strip(),
    ibm_crn := open(ENV['HOME'] + '/work/api-keys/IBM_CRN').readline().strip()
)

In [6]:
ibmq.printqpus()

ibmq QiskitBackend('ibm_kingston') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_fez') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_marrakesh') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_torino') 133qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_brisbane') 127qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_sherbrooke') 127qb DeviceStatus.ONLINE


In [7]:
ibm_qpu = ibmq.provider.get_device('ibm_kingston')
ibm_qpu.status()

<DeviceStatus.ONLINE: 'online'>

## Run circuits

Basic test run

In [12]:
# test run our jobs
if verify_run():
    job_test_amp_qft = ibm_qpu.run(amp_qft_circuits[:2], shots=SHOTS)
    job_test_amp     = ibm_qpu.run(amp_circuits[:2],     shots=SHOTS)
    job_test_ang     = ibm_qpu.run(ang_circuits[:2],     shots=SHOTS)
    # Based on our testing, angle embedding is the fastest

You must type "REALLY" to run this because it may cost a lot of money!!! >>>  REALLY
If you are really sure you want to run this code, type "REALLY" to proceed >>>  REALLY


In [ ]:
# run for real
if verify_run():
    job_test_ang = ibm_qpu.run(ang_circuits, shots=SHOTS)

In [10]:
job_test_amp_qft, job_test_amp, job_test_ang

(<RuntimeJobV2('d1pl8hn6d0hc73b8ds60', 'sampler')>,
 <RuntimeJobV2('d1pl8in6d0hc73b8ds70', 'sampler')>,
 <RuntimeJobV2('d1pl8j95jdrc73dst4j0', 'sampler')>)

In [11]:
job_test_amp_qft.wait_for_final_state()

In [12]:
ibmq.jobcircuit(ibmq.jobid(job_test_ang)).trim().draw(fold=-1)

┌──────────┐┌────┐┌────────────┐┌────┐                       ┌────┐  ┌─────────┐                     ┌─┐                           
 q109: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(2.7352) ├┤ √X ├──────────────■────────┤ √X ├──┤ Rz(π/2) ├─────────────────────┤M├───────────────────────────
       ├──────────┤├────┤├────────────┤├────┤   ┌────┐     │     ┌──┴────┴─┐└─────────┘┌─┐                  └╥┘                           
   q3: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(2.8691) ├┤ √X ├─■─┤ √X ├─────┼─────┤ Rz(π/2) ├───────────┤M├───────────────────╫────────────────────────────
       ├─────────┬┘├────┤├────────────┤└────┘ │ ├────┤     │     ├─────────┤           └╥┘                   ║ ┌─┐                        
   q2: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3.3885) ├───────■─┤ √X ├─────┼─────┤ Rz(π/2) ├────────────╫────────────────────╫─┤M├────────────────────────
       ├─────────┤ ├────┤├───────────┬┘         └────┘     │     └──┬────┬─┘┌─────────┐ ║                    ║ └╥┘         ┌─┐            
   q7: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3.382) ├────────────■────────┼────────┤ √X ├──┤ Rz(π/2) ├─╫────────────────────╫──╫──────────┤M├────────────
       ├─────────┴┐├────┤├───────────┴┐┌────┐     │        │        ├────┤  ├─────────┤ ║ ┌─┐                ║  ║          └╥┘            
   q4: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(2.9153) ├┤ √X ├─■───┼────────┼────────┤ √X ├──┤ Rz(π/2) ├─╫─┤M├────────────────╫──╫───────────╫─────────────
       ├─────────┬┘├────┤├────────────┤└────┘ │   │        │        ├────┤  ├─────────┤ ║ └╥┘                ║  ║ ┌─┐       ║             
   q5: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3.3934) ├───────■───┼────────┼────────┤ √X ├──┤ Rz(π/2) ├─╫──╫─────────────────╫──╫─┤M├───────╫─────────────
       ├─────────┴┐├────┤├────────────┤┌────┐     │        │        ├────┤  ├─────────┤ ║  ║    ┌─┐          ║  ║ └╥┘       ║             
   q6: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(2.6629) ├┤ √X ├─────■────────┼────────┤ √X ├──┤ Rz(π/2) ├─╫──╫────┤M├──────────╫──╫──╫────────╫─────────────
       ├──────────┤├────┤├────────────┤├────┤   ┌────┐     │     ┌──┴────┴─┐└─────────┘ ║  ║    └╥┘   ┌─┐    ║  ║  ║        ║             
  q24: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(2.7531) ├┤ √X ├─■─┤ √X ├─────┼─────┤ Rz(π/2) ├────────────╫──╫─────╫────┤M├────╫──╫──╫────────╫─────────────
       ├─────────┬┘├────┤├────────────┤└────┘ │ ├────┤     │     ├─────────┤            ║  ║     ║    └╥┘    ║  ║  ║        ║    ┌─┐      
  q25: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3.2937) ├───────■─┤ √X ├─────┼─────┤ Rz(π/2) ├────────────╫──╫─────╫─────╫─────╫──╫──╫────────╫────┤M├──────
       ├─────────┤ ├────┤├───────────┬┘         └────┘     │     └──┬────┬─┘┌─────────┐ ║  ║     ║     ║     ║  ║  ║        ║    └╥┘   ┌─┐
 q110: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3.501) ├─────────────────────■────────┤ √X ├──┤ Rz(π/2) ├─╫──╫─────╫─────╫─────╫──╫──╫────────╫─────╫────┤M├
       ├─────────┴┐├────┤├───────────┴┐┌────┐   ┌────┐┌─────────┐   └────┘  └─────────┘ ║  ║     ║ ┌─┐ ║     ║  ║  ║        ║     ║    └╥┘
  q22: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(2.9253) ├┤ √X ├─■─┤ √X ├┤ Rz(π/2) ├───────────────────────╫──╫─────╫─┤M├─╫─────╫──╫──╫────────╫─────╫─────╫─
       ├─────────┬┘├────┤├────────────┤└────┘ │ ├────┤├─────────┤                       ║  ║     ║ └╥┘ ║     ║  ║  ║        ║ ┌─┐ ║     ║ 
  q21: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3.8194) ├───────■─┤ √X ├┤ Rz(π/2) ├───────────────────────╫──╫─────╫──╫──╫─────╫──╫──╫────────╫─┤M├─╫─────╫─
       ├─────────┤ ├────┤├────────────┤         └────┘└──┬────┬─┘┌─────────┐            ║  ║     ║  ║  ║     ║  ║  ║ ┌─┐    ║ └╥┘ ║     ║ 
  q23: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3.4029) ├───────────■──────┤ √X ├──┤ Rz(π/2) ├────────────╫──╫─────╫──╫──╫─────╫──╫──╫─┤M├────╫──╫──╫─────╫─
       ├─────────┴┐├────┤├────────────┤┌────┐     │      ├────┤  ├─────────┤            ║  ║ ┌─┐ ║  ║  ║     ║  ║  ║ └╥┘    ║  ║  ║     ║ 
   q1: ┤ Rz(-π/2) ├┤ √X ├┤ Rz(2.7995) ├┤ √X ├─■───┼──────┤ √X ├──┤ Rz(π/2) ├────────────╫──╫─┤M├─╫──╫──╫─────╫──╫──╫──╫─────╫──╫──╫─────╫─
       ├─────────┬┘├────┤├───────────┬┘└────┘ │   │      ├────┤  ├─────────┤            ║  ║ └╥┘ ║  ║  ║     ║  ║  

In [13]:
ibmq.jobcircuit(ibmq.jobid(job_test_amp)).trim().draw(fold=-1)

┌────────┐     ┌────┐     ┌─────────────┐                                                                    ┌────┐   ┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         ┌────┐┌──────────────┐┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      ┌───┐      ┌─────────┐                                                                                                             ┌───┐    ┌──────────┐                         ┌────┐  ┌─────────────┐    ┌────┐    ┌────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                               ┌────┐                                                            ┌────┐   ┌────┐        ┌─┐                                                                                                     
q104: ┤ Rz(-π) ├─────┤ √X ├─────┤ Rz(-3.0848) ├────────────────────────────────────────────────────────────────■───┤ √X ├─■─┤ √X ├─■─────────────────────────────────────────────────────────────────────■────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────┤ √X ├┤ Rz(0.097501) ├┤ √X ├─────────────────────────────────────────────────────■──────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────■──────┤ X ├──────┤ Rz(π/2) ├─────

In [14]:
ibmq.jobcircuit(ibmq.jobid(job_test_amp_qft)).trim().draw(fold=-1)

┌────────┐    ┌────┐    ┌───────────────┐               ┌────┐ ┌────────────┐┌────┐   ┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 ┌────┐  ┌────────┐                                                     ┌────┐      ┌────┐                              ┌──────────────┐   ┌────┐  ┌─────────┐                                                                                                                                                 ┌────┐   ┌────┐   ┌──────────┐┌────┐┌─────────────┐┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ┌────┐                          ┌────┐┌─────────────┐                                                                                                                                   ┌────┐                                               ┌────┐ ┌────────────┐┌────┐   ┌────────┐  ┌────┐                                                                                                                                                                ┌──────────┐    ┌────┐   ┌─────────────┐┌────┐                                            ┌────┐                 ┌────┐┌────────────┐                                                     ┌────┐                                                                    ┌────┐       ┌────┐   ┌─────────┐    ┌────┐    ┌───────┐                                                                                                               ┌────┐┌───────────┐┌────┐   ┌────┐┌───────────┐     ┌────┐                                                                ┌────┐┌───────────┐┌────┐┌────────────┐   ┌────┐┌───────────┐    ┌────┐   ┌────────────┐                         ┌────┐   ┌────┐                                                                                                                                                                                                                                                                                 ┌──────────────┐    ┌────┐    ┌────────┐                                                               ┌────┐┌──────────────┐   ┌────┐┌─────────────┐┌────┐┌────────────┐   ┌─┐                                                                                                                                                                                                                                                                                                         
q116: ┤ Rz(-π) ├────┤ √X ├────┤ Rz(-0.097501) ├─────────────■─┤ √X ├─┤ Rz(2.9552) ├┤ √X ├─■─┤ √X ├───────────────────────────────────────────────────────────────────────■────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─┤ √X ├──┤ Rz(-π) ├───────────────────────────────────────────────────■─┤ √X ├─■────┤ √X ├────────────────────────────■─┤ Rz(-0.08653) ├───┤ √X ├──┤ Rz(π/2) ├────────────────

In [15]:
res = job_test_ang.result()

In [18]:
counts = ibmq.jobresult(ibmq.jobid(job_test_ang))

In [21]:
swap_expectation({int(k,2):v/(2**18) for k,v in counts.items()}, size=9)

(np.float64(0.00278472900390625), np.float64(0.5013923645019531))

In [43]:
len(ang_circuits)

2926

In [83]:
def stati(jobs:dict[int,any]) -> dict[int,any]:
    return {k:v.status() for k,v in jobs.items()}

In [85]:
# run for real
# may have to batch values
if verify_run():
    ANSWER_ang = {
        i: ibm_qpu.run(ang_circuits_chunk, shots=SHOTS)
        for i, ang_circuits_chunk in tqdm(chunk(ang_circuits, 50).items())
    }

stati(ANSWER_ang)

You must type "REALLY" to run this because it may cost a lot of money!!! >>>  no


By not typing "REALLY", you did not run this code


{0: <COMPLETED: 'job has successfully run'>,
 1: <COMPLETED: 'job has successfully run'>,
 2: <COMPLETED: 'job has successfully run'>,
 3: <COMPLETED: 'job has successfully run'>,
 4: <COMPLETED: 'job has successfully run'>,
 5: <COMPLETED: 'job has successfully run'>,
 6: <COMPLETED: 'job has successfully run'>,
 7: <COMPLETED: 'job has successfully run'>,
 8: <COMPLETED: 'job has successfully run'>,
 9: <COMPLETED: 'job has successfully run'>,
 10: <COMPLETED: 'job has successfully run'>,
 11: <COMPLETED: 'job has successfully run'>,
 12: <COMPLETED: 'job has successfully run'>,
 13: <COMPLETED: 'job has successfully run'>,
 14: <COMPLETED: 'job has successfully run'>,
 15: <COMPLETED: 'job has successfully run'>,
 16: <COMPLETED: 'job has successfully run'>,
 17: <COMPLETED: 'job has successfully run'>,
 18: <COMPLETED: 'job has successfully run'>,
 19: <COMPLETED: 'job has successfully run'>,
 20: <COMPLETED: 'job has successfully run'>,
 21: <COMPLETED: 'job has successfully run'>

In [202]:
# we had one job die, so we resubmitted it
ANSWER_ang_fixed = ANSWER_ang.copy()
ANSWER_ang_fixed[55] = ANSWER_ang_55

In [84]:
stati(ANSWER_ang_fixed)

{0: <COMPLETED: 'job has successfully run'>,
 1: <COMPLETED: 'job has successfully run'>,
 2: <COMPLETED: 'job has successfully run'>,
 3: <COMPLETED: 'job has successfully run'>,
 4: <COMPLETED: 'job has successfully run'>,
 5: <COMPLETED: 'job has successfully run'>,
 6: <COMPLETED: 'job has successfully run'>,
 7: <COMPLETED: 'job has successfully run'>,
 8: <COMPLETED: 'job has successfully run'>,
 9: <COMPLETED: 'job has successfully run'>,
 10: <COMPLETED: 'job has successfully run'>,
 11: <COMPLETED: 'job has successfully run'>,
 12: <COMPLETED: 'job has successfully run'>,
 13: <COMPLETED: 'job has successfully run'>,
 14: <COMPLETED: 'job has successfully run'>,
 15: <COMPLETED: 'job has successfully run'>,
 16: <COMPLETED: 'job has successfully run'>,
 17: <COMPLETED: 'job has successfully run'>,
 18: <COMPLETED: 'job has successfully run'>,
 19: <COMPLETED: 'job has successfully run'>,
 20: <COMPLETED: 'job has successfully run'>,
 21: <COMPLETED: 'job has successfully run'>

In [80]:
amp_qft_ddd_circuits_df = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed_qft_ddd.xlsx', index_col=0)
amp_qft_ddd_circuits  = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_qft_ddd_circuits_df['qasm2_ddd'])]
for i,c in enumerate(amp_qft_ddd_circuits): c.name = f'amp_qft_ddd_c{i}'

  0%|          | 0/2926 [00:00<?, ?it/s]

In [92]:
if verify_run():
    ANSWER_amp_qft_ddd = {
        i: ibm_qpu.run(amp_qft_ddd_circuits_chunk, shots=SHOTS)
        for i, amp_qft_ddd_circuits_chunk in tqdm(chunk(amp_qft_ddd_circuits, 45).items())
    }
    print(stati(ANSWER_amp_qft_ddd))

    ANSWER_amp_qft = {
        i: ibm_qpu.run(amp_qft_circuits_chunk, shots=SHOTS)
        for i, amp_qft_circuits_chunk in tqdm(chunk(amp_qft_circuits, 40).items())
    }
    print(stati(ANSWER_amp_qft))


You must type "REALLY" to run this because it may cost a lot of money!!! >>>  REALLY
If you are really sure you want to run this code, type "REALLY" to proceed >>>  REALLY


  0%|          | 0/66 [00:00<?, ?it/s]

{0: <COMPLETED: 'job has successfully run'>, 1: <COMPLETED: 'job has successfully run'>, 2: <COMPLETED: 'job has successfully run'>, 3: <COMPLETED: 'job has successfully run'>, 4: <COMPLETED: 'job has successfully run'>, 5: <COMPLETED: 'job has successfully run'>, 6: <COMPLETED: 'job has successfully run'>, 7: <COMPLETED: 'job has successfully run'>, 8: <COMPLETED: 'job has successfully run'>, 9: <COMPLETED: 'job has successfully run'>, 10: <COMPLETED: 'job has successfully run'>, 11: <COMPLETED: 'job has successfully run'>, 12: <COMPLETED: 'job has successfully run'>, 13: <COMPLETED: 'job has successfully run'>, 14: <COMPLETED: 'job has successfully run'>, 15: <COMPLETED: 'job has successfully run'>, 16: <COMPLETED: 'job has successfully run'>, 17: <COMPLETED: 'job has successfully run'>, 18: <COMPLETED: 'job has successfully run'>, 19: <COMPLETED: 'job has successfully run'>, 20: <COMPLETED: 'job has successfully run'>, 21: <COMPLETED: 'job has successfully run'>, 22: <COMPLETED: 'jo

  0%|          | 0/74 [00:00<?, ?it/s]

{0: <COMPLETED: 'job has successfully run'>, 1: <COMPLETED: 'job has successfully run'>, 2: <COMPLETED: 'job has successfully run'>, 3: <COMPLETED: 'job has successfully run'>, 4: <COMPLETED: 'job has successfully run'>, 5: <COMPLETED: 'job has successfully run'>, 6: <COMPLETED: 'job has successfully run'>, 7: <COMPLETED: 'job has successfully run'>, 8: <COMPLETED: 'job has successfully run'>, 9: <COMPLETED: 'job has successfully run'>, 10: <COMPLETED: 'job has successfully run'>, 11: <COMPLETED: 'job has successfully run'>, 12: <COMPLETED: 'job has successfully run'>, 13: <COMPLETED: 'job has successfully run'>, 14: <COMPLETED: 'job has successfully run'>, 15: <COMPLETED: 'job has successfully run'>, 16: <COMPLETED: 'job has successfully run'>, 17: <COMPLETED: 'job has successfully run'>, 18: <COMPLETED: 'job has successfully run'>, 19: <COMPLETED: 'job has successfully run'>, 20: <COMPLETED: 'job has successfully run'>, 21: <COMPLETED: 'job has successfully run'>, 22: <COMPLETED: 'jo

In [184]:
amp_ddd_circuits_df = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed_ddd.xlsx', index_col=0)
amp_ddd_circuits  = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_ddd_circuits_df['qasm2_ddd'])]
for i,c in enumerate(amp_ddd_circuits): c.name = f'amp_ddd_c{i}'

  0%|          | 0/2926 [00:00<?, ?it/s]

In [187]:
if verify_run():
    ANSWER_amp_ddd = {
        i: ibm_qpu.run(amp_ddd_circuits_chunk, shots=SHOTS)
        for i, amp_ddd_circuits_chunk in tqdm(chunk(amp_ddd_circuits, 42).items())
    }
    print(stati(ANSWER_amp_ddd))

    ANSWER_amp = {
        i: ibm_qpu.run(amp_circuits_chunk, shots=SHOTS)
        for i, amp_circuits_chunk in tqdm(chunk(amp_circuits, 47).items())
    }
    print(stati(ANSWER_amp))


You must type "REALLY" to run this because it may cost a lot of money!!! >>>  REALLY
If you are really sure you want to run this code, type "REALLY" to proceed >>>  REALLY


  0%|          | 0/70 [00:00<?, ?it/s]

{0: <COMPLETED: 'job has successfully run'>, 1: <COMPLETED: 'job has successfully run'>, 2: <COMPLETED: 'job has successfully run'>, 3: <COMPLETED: 'job has successfully run'>, 4: <COMPLETED: 'job has successfully run'>, 5: <COMPLETED: 'job has successfully run'>, 6: <COMPLETED: 'job has successfully run'>, 7: <COMPLETED: 'job has successfully run'>, 8: <COMPLETED: 'job has successfully run'>, 9: <COMPLETED: 'job has successfully run'>, 10: <COMPLETED: 'job has successfully run'>, 11: <COMPLETED: 'job has successfully run'>, 12: <COMPLETED: 'job has successfully run'>, 13: <COMPLETED: 'job has successfully run'>, 14: <COMPLETED: 'job has successfully run'>, 15: <COMPLETED: 'job has successfully run'>, 16: <COMPLETED: 'job has successfully run'>, 17: <COMPLETED: 'job has successfully run'>, 18: <COMPLETED: 'job has successfully run'>, 19: <COMPLETED: 'job has successfully run'>, 20: <COMPLETED: 'job has successfully run'>, 21: <COMPLETED: 'job has successfully run'>, 22: <COMPLETED: 'jo

  0%|          | 0/63 [00:00<?, ?it/s]

{0: <COMPLETED: 'job has successfully run'>, 1: <COMPLETED: 'job has successfully run'>, 2: <COMPLETED: 'job has successfully run'>, 3: <COMPLETED: 'job has successfully run'>, 4: <COMPLETED: 'job has successfully run'>, 5: <COMPLETED: 'job has successfully run'>, 6: <COMPLETED: 'job has successfully run'>, 7: <COMPLETED: 'job has successfully run'>, 8: <COMPLETED: 'job has successfully run'>, 9: <COMPLETED: 'job has successfully run'>, 10: <COMPLETED: 'job has successfully run'>, 11: <COMPLETED: 'job has successfully run'>, 12: <COMPLETED: 'job has successfully run'>, 13: <COMPLETED: 'job has successfully run'>, 14: <COMPLETED: 'job has successfully run'>, 15: <COMPLETED: 'job has successfully run'>, 16: <COMPLETED: 'job has successfully run'>, 17: <COMPLETED: 'job has successfully run'>, 18: <COMPLETED: 'job has successfully run'>, 19: <COMPLETED: 'job has successfully run'>, 20: <COMPLETED: 'job has successfully run'>, 21: <COMPLETED: 'job has successfully run'>, 22: <COMPLETED: 'jo

In [191]:
print(stati(ANSWER_amp_ddd))

{0: <COMPLETED: 'job has successfully run'>, 1: <COMPLETED: 'job has successfully run'>, 2: <COMPLETED: 'job has successfully run'>, 3: <COMPLETED: 'job has successfully run'>, 4: <COMPLETED: 'job has successfully run'>, 5: <COMPLETED: 'job has successfully run'>, 6: <COMPLETED: 'job has successfully run'>, 7: <COMPLETED: 'job has successfully run'>, 8: <COMPLETED: 'job has successfully run'>, 9: <COMPLETED: 'job has successfully run'>, 10: <COMPLETED: 'job has successfully run'>, 11: <COMPLETED: 'job has successfully run'>, 12: <COMPLETED: 'job has successfully run'>, 13: <COMPLETED: 'job has successfully run'>, 14: <COMPLETED: 'job has successfully run'>, 15: <COMPLETED: 'job has successfully run'>, 16: <COMPLETED: 'job has successfully run'>, 17: <COMPLETED: 'job has successfully run'>, 18: <COMPLETED: 'job has successfully run'>, 19: <COMPLETED: 'job has successfully run'>, 20: <COMPLETED: 'job has successfully run'>, 21: <COMPLETED: 'job has successfully run'>, 22: <COMPLETED: 'jo

In [192]:
print(stati(ANSWER_amp))

{0: <COMPLETED: 'job has successfully run'>, 1: <COMPLETED: 'job has successfully run'>, 2: <COMPLETED: 'job has successfully run'>, 3: <COMPLETED: 'job has successfully run'>, 4: <COMPLETED: 'job has successfully run'>, 5: <COMPLETED: 'job has successfully run'>, 6: <COMPLETED: 'job has successfully run'>, 7: <COMPLETED: 'job has successfully run'>, 8: <COMPLETED: 'job has successfully run'>, 9: <COMPLETED: 'job has successfully run'>, 10: <COMPLETED: 'job has successfully run'>, 11: <COMPLETED: 'job has successfully run'>, 12: <COMPLETED: 'job has successfully run'>, 13: <COMPLETED: 'job has successfully run'>, 14: <COMPLETED: 'job has successfully run'>, 15: <COMPLETED: 'job has successfully run'>, 16: <COMPLETED: 'job has successfully run'>, 17: <COMPLETED: 'job has successfully run'>, 18: <COMPLETED: 'job has successfully run'>, 19: <COMPLETED: 'job has successfully run'>, 20: <COMPLETED: 'job has successfully run'>, 21: <COMPLETED: 'job has successfully run'>, 22: <COMPLETED: 'jo

print list of jobs to file

In [193]:
with open(ENV['HOME'] + '/work/jobs.json', 'wt+') as f:
    json.dump({
        'ang': {k:v.id for k,v in ANSWER_ang_fixed.items()},
        'amp_qft_ddd': {k:v.id for k,v in ANSWER_amp_qft_ddd.items()},
        'amp_qft': {k:v.id for k,v in ANSWER_amp_qft.items()},
        'amp': {k:v.id for k,v in ANSWER_amp.items()},
        'amp_ddd': {k:v.id for k,v in ANSWER_amp_ddd.items()},
    }, fp=f, indent=4)